# Code for AISTATS submission (Reconstructing Test Labels from Noisy Loss Functions)
## Label inference on log-loss, sigmoid-loss, and softmax-loss

### Experiment setup

In [ ]:
import os
import json
import sys
import pandas as pd
import numpy as np
from collections import Counter
import math
import time
import warnings
warnings.filterwarnings('ignore')

np.random.seed(12345)

### Extract labels from selected openML multi-class classification datasets
1. titanic: https://www.openml.org/d/40704
2. iris: https://www.openml.org/d/41511
3. satellite image: https://www.openml.org/d/182

In [ ]:
all_labels = dict() # a dictionary to hold all labels from real datasets
os.listdir('data')

#### 1. titanic

In [ ]:
titanic = pd.read_csv('data/titanic.csv')
titanic = titanic.sample(frac=1)
print(titanic.shape) 
print(Counter(titanic['class']))
titanic.head()

In [ ]:
def titanic_to_int(x):
    return x if x == 1 else 0
titanic_label = titanic['class'].apply(titanic_to_int)
all_labels['titanic'] = titanic_label.tolist()
Counter(titanic_label)

#### 2. iris

In [ ]:
iris = pd.read_csv('data/iris.csv')
iris = iris.sample(frac=1)
print(iris.shape) 
print(Counter(iris['Species']))
iris.head()

In [ ]:
def iris_to_int(txt):
    if txt == 'setosa':
        return 0
    if txt == 'versicolor':
        return 1
    if txt == 'virginica':
        return 2
iris_label = iris['Species'].apply(iris_to_int)
all_labels['iris'] = iris_label.tolist()
Counter(iris_label)

#### 3. satellite image

In [ ]:
satimage = pd.read_csv('data/satimage.csv')
satimage = satimage.sample(frac=1)
print(satimage.shape)
print(Counter(satimage['class']))
satimage.head()

In [ ]:
def satimage_to_int(txt):
    mapping = {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4, 7.0: 5}
    return mapping[txt]
satimage_label = satimage['class'].apply(satimage_to_int)
all_labels['satimage'] = satimage_label.tolist()
Counter(satimage_label)

## utility functions in the experiments

In [ ]:
# function to compute the k-ary-logloss score
def k_ary_logloss(labeling=[], prediction=np.array([[]])):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the k-ary-logloss score
    """
    N = len(labeling)
    loss = 0
    for i in range(N):
        cur_pred = prediction[i, labeling[i]]
        loss -= math.log(cur_pred)
    return loss / N

In [ ]:
# function to compute the sigmoid loss
def sigmoid_loss(labeling=[], prediction=np.array([[]])):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the sigmoid loss score
    """
    N = len(labeling)
    loss = 0
    for i in range(N):
        cur_pred = prediction[i, labeling[i]]
        sig = 1 / (1 + np.exp(-cur_pred))
        loss -= math.log(sig)
    return loss / N

In [ ]:
# function to compute the sigmoid loss
def softmax_loss(labeling=[], prediction=np.array([[]])):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the sigmoid loss score
    """
    N = len(labeling)
    loss = 0
    for i in range(N):
        cur_pred = prediction[i, labeling[i]]
        cur_num = np.exp(cur_pred)
        cur_den = np.sum(np.exp(prediction[i, :]))
        cur_softmax = cur_num / cur_den
        loss -= math.log(cur_softmax)
    return loss / N

In [ ]:
# wrapper loss function
def loss(labeling=[], prediction=np.array([[]]), tau=0, loss_type='logloss'):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    tau: noise
    loss_type: logloss, sigmoidloss, or softmaxloss
    Returns the loss score
    """
    if loss_type == 'logloss':
        return k_ary_logloss(labeling, prediction) + tau * np.random.rand()
    if loss_type == 'sigmoidloss':
        return sigmoid_loss(labeling, prediction) + tau * np.random.rand()
    if loss_type == 'softmaxloss':
        return softmax_loss(labeling, prediction) + tau * np.random.rand()

In [ ]:
# function to generate the predictions
def generate_prediction(N, K, tau_max):
    """
    N: number of data points
    K: number of classes
    tau_max: maximum noise
    Return a 2-d numpy array of predictions
    """
    prediction = np.zeros([N, K])
    for i in range(N):
        n = i + 1
        for j in range(K):
            k = j + 1
            prediction[i, j] = 2 ** ((n-1) * K + k) * N * tau_max
        
        diff = prediction[i, :] - max(prediction[i, :])
        prediction[i, :] = 3 ** diff
        prediction[i, :] /= sum(prediction[i, :])
        for j in range(K):
            prediction[i, j] = sys.float_info.min if prediction[i, j] < sys.float_info.min \
                else prediction[i, j]
    return prediction 

In [ ]:
# generator for K-nary labelings
def gen_K_ary(nbits, K):
    """
    nbits: number of bits in the K-nary vector
    K: K in K-nary, e.g. K=2 is binary, K=3 is ternary
    Returns a generator for all K-nary labelings of length nbits
    """
    for i in range(K**nbits):
        bin_str = str(np.base_repr(i, K))
        l = len(bin_str)
        bin_arr = np.zeros(nbits)
        for j in range(l):
            bin_arr[nbits-j-1] = float(bin_str[l-j-1])
        yield [int(x) for x in bin_arr]

In [ ]:
# the main label inference algorithm
def main_algorithm(noisy_loss, N, K, prediction, loss_type):
    """
    noisy_loss: observed loss score with noise
    N: number of data points
    K: number of classes
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    loss_type: logloss, sigmoidloss, or softmaxloss
    Returns the inferred labeling
    """
    label_infer = []
    diff_min = np.inf
    lab_generator = gen_K_ary(N, K)
    
    while True:
        try:
            label_cur = next(lab_generator) 
            loss_cur = loss(label_cur, prediction, 0, loss_type)
            diff_cur = np.abs(loss_cur - noisy_loss)
            # print(loss_cur, diff_cur)
            if diff_cur < diff_min:
                label_infer = label_cur.copy()
                diff_min = diff_cur
        except:
            break
    return label_infer

In [ ]:
# the main function for simulation
def main_sim(true_labeling, N, K, tau_max, loss_type, nsims=1000):
    """
    true_labeling: ground truth labeling with length at least N
    N: number of data points
    K: number of classes
    tau_max: maximum noise
    loss_type: logloss, sigmoidloss, or softmaxloss
    nsims: number of simulations
    Returns the average accuracy and the average loss score
    """
    acc = 0
    mean_loss = 0
    for ns in range(nsims):
        labeling = list(np.random.choice(true_labeling, N))
        tau = 2 * np.exp(1) * tau_max / (np.exp(1) - 1)
        prediction = generate_prediction(N, K, tau)
        noisy_loss = loss(labeling, prediction, tau_max, loss_type)
        mean_loss += noisy_loss
        labeling_infer = main_algorithm(noisy_loss, N, K, prediction, loss_type)
        all_same = (sum([x == y for x, y in zip(labeling_infer, labeling)]) == N)
        acc += all_same
    return acc / nsims, mean_loss / nsims

## label inference experiments

In [ ]:
# logloss experiment
loss_type = 'logloss'
nsims = 1000
result_logloss = []

for K in [2, 3, 6]:
    if K == 2:
        data = 'titanic'
    elif K == 3:
        data = 'iris'
    elif K == 6:
        data = 'satimage'

    for tau_max in [1e-4, 1e-2, 1e-1, 1]:
        for N in range(1, 16):
            try:
                start = time.time()
                acc, mean_loss = main_sim(all_labels[data], N, K, tau_max, loss_type, nsims)
                seconds = time.time() - start
                result_logloss.append((K, tau_max, N, acc, mean_loss, seconds))
                print(K, tau_max, N, acc, mean_loss, seconds)
                if acc < 0.5: break
            except:
                break

In [ ]:
df_logloss = pd.DataFrame(result_logloss)
df_logloss.columns = ['K', 'tau_max', 'N', 'acc', 'mean_loss', 'seconds_1k']
df_logloss['ms'] = df_logloss['seconds_1k'] # since 1000 repetitions
df_logloss.to_csv('logloss.csv', index=False)
print(df_logloss.shape)
df_logloss.head(10)

In [ ]:
# sigmoid loss experiment
loss_type = 'sigmoidloss'
nsims = 1000
result_sigmoidloss = []

for K in [2, 3, 6]:
    if K == 2:
        data = 'titanic'
    elif K == 3:
        data = 'iris'
    elif K == 6:
        data = 'satimage'

    for tau_max in [1e-4, 1e-2, 1e-1, 1]:
        for N in range(1, 16):
            try:
                start = time.time()
                acc, mean_loss = main_sim(all_labels[data], N, K, tau_max, loss_type, nsims)
                seconds = time.time() - start
                result_sigmoidloss.append((K, tau_max, N, acc, mean_loss, seconds))
                print(K, tau_max, N, acc, mean_loss, seconds)
                if acc < 0.5: break
            except:
                break

In [ ]:
df_sigmoidloss = pd.DataFrame(result_sigmoidloss)
df_sigmoidloss.columns = ['K', 'tau_max', 'N', 'acc', 'mean_loss', 'seconds_1k']
df_sigmoidloss['ms'] = df_sigmoidloss['seconds_1k'] # since 1000 repetitions
df_sigmoidloss.to_csv('sigmoidloss.csv', index=False)
print(df_sigmoidloss.shape)
df_sigmoidloss.head(10)

In [ ]:
# softmax loss experiment
loss_type = 'softmaxloss'
nsims = 1000
result_softmaxloss = []

for K in [2, 3, 6]:
    if K == 2:
        data = 'titanic'
    elif K == 3:
        data = 'iris'
    elif K == 6:
        data = 'satimage'

    for tau_max in [1e-4, 1e-2, 1e-1, 1]:
        for N in range(1, 16):
            try:
                start = time.time()
                acc, mean_loss = main_sim(all_labels[data], N, K, tau_max, loss_type, nsims)
                seconds = time.time() - start
                result_softmaxloss.append((K, tau_max, N, acc, mean_loss, seconds))
                print(K, tau_max, N, acc, mean_loss, seconds)
                if acc < 0.5: break
            except:
                break

In [ ]:
df_softmaxloss = pd.DataFrame(result_softmaxloss)
df_softmaxloss.columns = ['K', 'tau_max', 'N', 'acc', 'mean_loss', 'seconds_1k']
df_softmaxloss['ms'] = df_softmaxloss['seconds_1k'] # since 1000 repetitions
df_softmaxloss.to_csv('softmaxloss.csv', index=False)
print(df_softmaxloss.shape)
df_softmaxloss.head(10)